In [ ]:
import json
import os
import re
from datetime import datetime, timedelta
from collections import defaultdict
from decimal import Decimal

import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.io import to_html

import yfinance as yf

from IPython.display import display, HTML

from transaction_history_processor import PortfolioHistory

In [ ]:
if not os.path.exists('portfolio_history.json'):
    print("portfolio_history.json not found. Generating it now...")
    portfolio_history = PortfolioHistory()
    portfolio_history.process_transaction_history(
        input_file='transaction_history.json',
        save_output=True,
        output_file='portfolio_history.json'
    )
    print("portfolio_history.json has been generated.")

with open('portfolio_history.json', 'r') as f:
    data = json.load(f)
    sectors = data['sectors']
    portfolio_data = data['portfolios']

In [ ]:
def plot_stock_proportions(portfolio_data, date=None, show_values=True, main_threshold=3.0):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return None

    holdings = portfolio_data[date]['holdings']
    labels = list(holdings.keys())
    values = [float(data['market_value']) for data in holdings.values()]

    total = sum(values)
    percentages = [(value / total) * 100 for value in values]

    main_holdings = list(zip(labels, values, percentages))
    main_holdings.sort(key=lambda x: x[2], reverse=True)

    other_holdings = [h for h in main_holdings if h[2] < main_threshold]
    if len(other_holdings) > 1:
        others_value = sum(v for _, v, _ in other_holdings)
        others_percentage = sum(p for _, _, p in other_holdings)
        other_symbols_info = "<br>".join(f"<span style='font-family: monospace;'>{l:<5}\t{
                                         p:>6.2f}%</span>" for l, _, p in other_holdings)
        main_holdings = [h for h in main_holdings if h[2] >=
                         main_threshold] + [("Others", others_value, others_percentage)]

    main_labels, main_values, main_percentages = zip(*main_holdings)

    # Create custom hover text
    hover_text = [f"{label}<br>${value:,.2f}<br>{percentage:.2f}%" for label,
                  value, percentage in zip(main_labels, main_values, main_percentages)]
    if "Others" in main_labels:
        others_index = main_labels.index("Others")
        hover_text[others_index] += f"<br><br>{other_symbols_info}"

    fig = go.Figure(data=[go.Pie(labels=main_labels,
                    values=main_percentages,
                    textinfo='label+percent',
                    hovertext=hover_text,
                    hoverinfo='text')])

    fig.update_layout(
        title=f'Stock Proportions by Value on {date}',
        showlegend=True,
        legend=dict(title="Holdings")
    )

    if show_values:
        fig.update_traces(
            text=[f'${value:,.2f}' for value in main_values], textposition='inside')

    return fig

# Call the function
plot_stock_proportions(portfolio_data, show_values=False)

In [ ]:
# plot_stock_proportions(
#     portfolio_history, date="2023-10-26")

In [ ]:
def plot_sector_proportions(portfolio_data, sectors, date=None):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return None

    holdings = portfolio_data[date]['holdings']
    sector_counts = {}
    for symbol, data in holdings.items():
        sector = sectors.get(symbol, 'ETF')
        if sector not in sector_counts:
            sector_counts[sector] = 0
        sector_counts[sector] += float(data['quantity']) * \
            float(data['market_price'])

    labels = list(sector_counts.keys())
    sizes = list(sector_counts.values())

    fig = go.Figure(
        data=[go.Pie(labels=labels, values=sizes, textinfo='label+percent')])

    fig.update_layout(
        title=f'Sector Proportions on {date}',
        showlegend=True,
        legend=dict(title="Sectors")
    )

    return fig

# Call the function with the loaded data
plot_sector_proportions(portfolio_data, sectors)

In [ ]:
def plot_portfolio_growth(portfolio_data):
    dates = sorted(portfolio_data.keys())
    start_date = pd.to_datetime(dates[0])
    dates = [d for d in dates if pd.to_datetime(d) >= start_date]

    df_plot = pd.DataFrame({
        'Date': pd.to_datetime(dates),
        'Total Market Value': [float(portfolio_data[d]['summary']['total_market_value']) for d in dates],
        'Cash': [float(portfolio_data[d]['summary']['cash']) for d in dates],
        'Unrealized Gain': [float(portfolio_data[d]['summary']['unrealized_gain_loss']) for d in dates],
        'Daily Gain': [float(portfolio_data[d]['summary']['daily_gain']) for d in dates],
        'Cumulative Return (%)': [float(portfolio_data[d]['summary']['cumulative_return']) * 100 for d in dates],
        'ROI (%)': [float(portfolio_data[d]['summary']['roi']) * 100 for d in dates],
    })

    latest_date = df_plot['Date'].max()
    returns = calculate_returns(df_plot, latest_date)

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    add_sp500_trace(fig, df_plot)
    add_portfolio_traces(fig, df_plot)
    add_performance_annotation(fig, returns)

    # Apply layout updates
    fig.update_layout(
        title={'text': 'Portfolio Performance & Value vs S&P 500', 'y': 0.95, 'x': 0.5,
               'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title='Date',
        hovermode='x unified',
        showlegend=True,
        xaxis=dict(
            rangeslider=dict(visible=False),
            rangeselector=dict(
                buttons=list([
                    dict(count=5, label="5D", step="day", stepmode="backward"),
                    dict(count=1, label="1M", step="month", stepmode="backward"),
                    dict(count=6, label="6M", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1Y", step="year", stepmode="backward"),
                    dict(step="all", label="MAX")
                ])
            )
        ),
        margin=dict(t=120),
    )
    fig.update_yaxes(title_text="Performance (%)",
                     ticksuffix='%', secondary_y=False)
    fig.update_yaxes(title_text="Value ($)", tickprefix='$', secondary_y=True)

    return fig

def calculate_returns(df_plot, latest_date):
    return {
        '5D': calculate_return(df_plot, latest_date, days=5),
        '1M': calculate_return(df_plot, latest_date, days=30),
        '6M': calculate_return(df_plot, latest_date, days=180),
        'YTD': calculate_return(df_plot, latest_date, ytd=True),
        '1Y': calculate_return(df_plot, latest_date, days=365),
        'All': df_plot['ROI (%)'].iloc[-1]
    }

def calculate_return(df_plot, latest_date, days=None, ytd=False):
    if ytd:
        start_date = pd.Timestamp(year=latest_date.year, month=1, day=1)
    else:
        start_date = latest_date - pd.Timedelta(days=days)
    return df_plot['ROI (%)'].iloc[-1] - df_plot[df_plot['Date'] >= start_date]['ROI (%)'].iloc[0]

def add_sp500_trace(fig, df_plot):
    sp500_data = fetch_sp500_data(df_plot['Date'].min(), df_plot['Date'].max())
    sp500_performance = (
        (sp500_data['Close'] - sp500_data['Close'].iloc[0]) / sp500_data['Close'].iloc[0]) * 100
    fig.add_trace(
        go.Scatter(x=sp500_data.index, y=sp500_performance.round(2), line=dict(color='blue', width=2),
                   name='S&P 500', visible=True,
                   hovertemplate='S&P 500: %{y:.2f}%<extra></extra>'),
        secondary_y=False
    )

def add_portfolio_traces(fig, df_plot):
    fig.add_trace(
        go.Scatter(x=df_plot['Date'], y=df_plot['ROI (%)'].round(2), fill='tozeroy',
                   fillcolor='rgba(255,0,0,0.2)', line=dict(color='red', width=2), name='ROI',
                   hovertemplate='ROI: %{y:.2f}% (Cum. Return: %{customdata:.2f}%)<extra></extra>',
                   customdata=df_plot['Cumulative Return (%)'].round(2), visible=True),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(x=df_plot['Date'], y=df_plot['Total Market Value'].round(2),
                   line=dict(color='green', width=2), name='Market Value', visible=True,
                   hovertemplate='$%{y:,.2f} (Cash: $%{customdata:,.2f})',
                   customdata=df_plot['Cash'].round(2)),
        secondary_y=True
    )
    fig.add_trace(
        go.Bar(x=df_plot['Date'], y=df_plot['Unrealized Gain'].round(2), name='Unr. Gain',
               marker_color='gray', opacity=0.9,
               hovertemplate='$%{y:,.2f} (Daily: $%{customdata:,.2f})',
               customdata=df_plot['Daily Gain'].round(2)),
        secondary_y=True
    )

def add_performance_annotation(fig, returns):
    performance_text = (
        f"Portfolio ROI:<br>"
        f"<span style='display: inline-block; width: 70px;'>5D: {returns['5D']:.2f}%</span> | "
        f"<span style='display: inline-block; width: 80px;'>1M: {returns['1M']:.2f}%</span><br>"
        f"<span style='display: inline-block; width: 70px;'>6M: {returns['6M']:.2f}%</span> | "
        f"<span style='display: inline-block; width: 80px;'>YTD: {returns['YTD']:.2f}%</span><br>"
        f"<span style='display: inline-block; width: 70px;'>1Y: {returns['1Y']:.2f}%</span> | "
        f"<span style='display: inline-block; width: 80px;'>All: {returns['All']:.2f}%</span>"
    )
    fig.add_annotation(
        x=1, y=1.05, xref='paper', yref='paper', text=performance_text, showarrow=False,
        font=dict(size=10, family="monospace"), align='left', xanchor='right', yanchor='bottom',
        bgcolor='rgba(255,255,255,0.8)', bordercolor='rgba(0,0,0,0.5)', borderwidth=1, borderpad=4
    )

def fetch_sp500_data(start_date, end_date):
    end_date = pd.to_datetime(end_date) + timedelta(days=1)
    sp500 = yf.Ticker("^GSPC")
    sp500_data = sp500.history(start=start_date, end=end_date)
    return sp500_data[['Close']]

In [ ]:
plot_portfolio_growth(portfolio_data)

In [ ]:
def process_portfolio_data(portfolio_data, date=None, sort_by='Symbol', ascending=True):
    date = date or max(portfolio_data.keys())
    if date not in portfolio_data:
        raise ValueError(f"No data available for date: {date}")

    data = portfolio_data[date]
    holdings_data = process_holdings(data['holdings'])

    df = pd.DataFrame(holdings_data)
    df = df.sort_values(by=sort_by, ascending=ascending)

    return {
        'date': date,
        'holdings': df,
        'summary': data['summary'],
        'realized_gains': data['summary'].get('realized_gains', 0),
    }


def process_holdings(holdings):
    return [
        {
            'Symbol': symbol,
            'Quantity': round(float(info['quantity']), 2),
            'Unit Cost': round(float(info['unit_cost']), 2),
            'Market Price': round(float(info['market_price']), 2),
            'Total Cost': round(float(info['total_cost']), 2),
            'Market Value': round(float(info['market_value']), 2),
            'Unrealized G/L': f"${float(info['unrealized_gain_loss']):,.2f}",
            'Daily Gain': f"${float(info['daily_gain']):,.2f} ({float(info['daily_return'])*100:.2f}%)"
        }
        for symbol, info in holdings.items()
    ]

In [ ]:
def display_portfolio_analysis(processed_data):
    date = processed_data['date']
    df = processed_data['holdings']
    summary = processed_data['summary']

    # Display holdings
    display(HTML(f"<h2>Portfolio Status as of {date}</h2>"))
    display(HTML(df.to_html(index=False, classes='dataframe')))

    # Display summary
    print("\nPortfolio Summary:")
    print_summary_item("Cash", summary.get('cash', 0))
    print_summary_item("Total Market Value",
                       summary.get('total_market_value', 0))
    print_summary_item("Unrealized Gain/Loss",
                       summary.get('unrealized_gain_loss', 0))
    print_summary_item("Realized Gains", processed_data['realized_gains'])
    print_summary_item("Daily Gain", summary.get('daily_gain', 0),
                       percentage=summary.get('daily_return', 0))
    print_summary_item("Total Return", summary.get(
        'total_return', 0), is_percentage=True)
    print_summary_item("Cumulative Return", summary.get(
        'cumulative_return', 0), is_percentage=True)
    print_summary_item("ROI", summary.get('roi', 0), is_percentage=True)


def print_summary_item(label, value, percentage=None, is_percentage=False):
    if is_percentage:
        print(f"{label}: {float(value)*100:.2f}%")
    elif percentage is not None:
        print(f"{label}: ${float(value):,.2f} ({float(percentage)*100:.2f}%)")
    else:
        print(f"{label}: ${float(value):,.2f}")

In [ ]:
processed_data = process_portfolio_data(portfolio_data)
display_portfolio_analysis(processed_data)

In [ ]:
# processed_data = process_portfolio_data(portfolio_data, date="2024-04-10")
# display_portfolio_analysis(processed_data)

In [ ]:
def process_transaction_history(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    positions = defaultdict(lambda: {
        'quantity': Decimal('0'),
        'cost': Decimal('0'),
        'total_sold': Decimal('0'),
        'total_proceeds': Decimal('0'),
        'last_sell_date': None
    })
    closed_positions = []

    for date, transactions in sorted(data.items()):
        for action in ['buy', 'sell']:
            if action in transactions:
                for symbol, trades in transactions[action].items():
                    for trade in trades:
                        quantity = Decimal(str(trade['quantity']))
                        amount = Decimal(str(trade['amount']))

                        if action == 'buy':
                            positions[symbol]['quantity'] += quantity
                            positions[symbol]['cost'] += amount
                        else:  # sell
                            sell_quantity = min(
                                quantity, positions[symbol]['quantity'])
                            positions[symbol]['quantity'] -= sell_quantity
                            positions[symbol]['total_sold'] += sell_quantity
                            positions[symbol]['total_proceeds'] += amount
                            positions[symbol]['last_sell_date'] = date

                            if positions[symbol]['quantity'] + sell_quantity > 0:
                                cost_basis_per_unit = positions[symbol]['cost'] / (
                                    positions[symbol]['quantity'] + sell_quantity)
                                profit_loss = amount - \
                                    (cost_basis_per_unit * sell_quantity)
                            else:
                                profit_loss = Decimal('0')
                            positions[symbol]['last_profit_loss'] = profit_loss

                            if positions[symbol]['quantity'] == Decimal('0'):
                                closed_positions.append({
                                    'symbol': symbol,
                                    'close_date': date,
                                    'quantity': positions[symbol]['total_sold'],
                                    'profit_loss': positions[symbol]['total_proceeds'] - positions[symbol]['cost']
                                })
                                del positions[symbol]

    return positions, closed_positions


def create_dataframes(positions, closed_positions):
    closed_positions_df = pd.DataFrame(closed_positions)

    partially_sold_positions = [
        {
            'symbol': symbol,
            'last_sell_date': data['last_sell_date'] or 'N/A',
            'remaining_quantity': data['quantity'],
            'total_sold': data['total_sold'],
            'profit_loss': data.get('last_profit_loss', Decimal('0'))
        }
        for symbol, data in positions.items() if data['total_sold'] > 0
    ]

    partially_sold_positions_df = pd.DataFrame(partially_sold_positions)
    partially_sold_positions_df['profit_loss'] = partially_sold_positions_df['profit_loss'].apply(
        lambda x: f"${x:.2f}")

    active_positions_df = partially_sold_positions_df.rename(columns={
        'last_sell_date': 'Last Sell Date',
        'remaining_quantity': 'Remaining Quantity',
        'total_sold': 'Total Sold',
        'profit_loss': 'Profit/Loss'
    })

    return closed_positions_df, active_positions_df

In [ ]:
positions, closed_positions = process_transaction_history(
    'transaction_history.json')
closed_positions_df, active_positions_df = create_dataframes(
    positions, closed_positions)

print("Closed Positions:")
display(closed_positions_df)

print("\nPartially Sold Positions:")
display(active_positions_df)

In [ ]:
def plot_asset_allocation_over_time(portfolio_data):
    dates = sorted(portfolio_data.keys())
    assets = get_unique_assets(portfolio_data)
    data = calculate_asset_percentages(portfolio_data, dates, assets)

    fig = go.Figure()
    add_asset_traces(fig, dates, data, assets)
    update_layout(fig)

    return fig


def get_unique_assets(portfolio_data):
    assets = set()
    for date_data in portfolio_data.values():
        assets.update(date_data['holdings'].keys())
    return sorted(list(assets))


def calculate_asset_percentages(portfolio_data, dates, assets):
    data = {asset: [] for asset in assets}
    for date in dates:
        total_value = sum(float(info['market_value'])
                          for info in portfolio_data[date]['holdings'].values())
        for asset in assets:
            percentage = calculate_asset_percentage(
                portfolio_data, date, asset, total_value)
            data[asset].append(percentage)
    return data


def calculate_asset_percentage(portfolio_data, date, asset, total_value):
    if asset in portfolio_data[date]['holdings']:
        value = float(portfolio_data[date]['holdings'][asset]['market_value'])
        return (value / total_value) * 100
    return 0


def add_asset_traces(fig, dates, data, assets):
    for asset in assets:
        fig.add_trace(go.Scatter(
            x=dates,
            y=data[asset],
            mode='lines',
            stackgroup='one',
            name=asset
        ))


def update_layout(fig):
    fig.update_layout(
        title='Asset Allocation Over Time',
        xaxis_title='Date',
        yaxis_title='Percentage',
        yaxis_tickformat=',.0%',
        legend_title='Assets',
        hovermode='x unified'
    )


# Call the function
plot_asset_allocation_over_time(portfolio_data)

In [ ]:
def generate_html_report():
    dates = sorted(portfolio_data.keys())
    latest_date = dates[-1]

    stock_proportions_plot = plot_stock_proportions(
        portfolio_data, latest_date)
    sector_proportions_plot = plot_sector_proportions(
        portfolio_data, sectors, latest_date)
    portfolio_growth_plot = plot_portfolio_growth(portfolio_data)
    asset_allocation_plot = plot_asset_allocation_over_time(portfolio_data)

    # Apply monospace font to all plots
    for plot in [stock_proportions_plot, sector_proportions_plot, portfolio_growth_plot, asset_allocation_plot]:
        plot.update_layout(font=dict(family="monospace", weight='bold'))

    processed_data = process_portfolio_data(portfolio_data, latest_date)

    positions, closed_positions = process_transaction_history(
        'transaction_history.json')
    closed_positions_df, active_positions_df = create_dataframes(
        positions, closed_positions)

    # Convert DataFrames to HTML strings
    closed_positions_table = closed_positions_df.to_html(
        index=False, classes='dataframe')
    partially_sold_positions_table = active_positions_df.to_html(
        index=False, classes='dataframe')
    portfolio_table = processed_data['holdings'].to_html(
        index=False, classes='dataframe')

    # Format portfolio summary
    summary = processed_data['summary']
    format_dict = {
        'date': latest_date,
        'portfolio_table': portfolio_table,
        'portfolio_growth_plot': to_html(portfolio_growth_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'stock_proportions_plot': to_html(stock_proportions_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'sector_proportions_plot': to_html(sector_proportions_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'asset_allocation_plot': to_html(asset_allocation_plot, full_html=False, include_plotlyjs=False, config={'responsive': True}),
        'date_options': ' '.join(f'<option value="{date}">{date}</option>' for date in dates),
        'cash': f"${float(summary.get('cash', 0)):,.2f}",
        'total_market_value': f"${float(summary.get('total_market_value', 0)):,.2f}",
        'unrealized_gain_loss': f"${float(summary.get('unrealized_gain_loss', 0)):,.2f}",
        'daily_gain': f"${float(summary.get('daily_gain', 0)):,.2f} ({float(summary.get('daily_return', 0))*100:.2f}%)",
        'total_realized_gains': f"${float(processed_data['realized_gains']):,.2f}",
        'total_return': f"{float(summary.get('total_return', 0))*100:.2f}%",
        'cumulative_return': f"{float(summary.get('cumulative_return', 0))*100:.2f}%",
        'roi': f"{float(summary.get('roi', 0))*100:.2f}%",
        'closed_positions_table': closed_positions_table,
        'partially_sold_positions_table': partially_sold_positions_table,
    }

    # Read the HTML template
    template_path = 'reports/templates/portfolio_report_template.html'
    with open(template_path, 'r') as template_file:
        html_template = template_file.read()

    # Read the CSS content
    css_path = 'reports/templates/portfolio_report_styles.css'
    with open(css_path, 'r') as css_file:
        css_content = css_file.read()

    # Escape curly braces in the CSS content
    css_content_escaped = css_content.replace('{', '{{').replace('}', '}}')

    # Replace the CSS placeholder with the escaped CSS content
    html_template = html_template.replace(
        '/* CSS_PLACEHOLDER */', css_content_escaped)

    # Remove whitespace from placeholders
    html_template = re.sub(r'\{\s*(\w+)\s*\}', r'{\1}', html_template)

    # Generate the HTML content
    html_content = html_template.format(**format_dict)

    # Ensure the 'reports' directory exists
    os.makedirs('reports', exist_ok=True)

    # Generate only the 'latest' version
    today_date = datetime.now().strftime("%Y-%m-%d")
    filename = f'portfolio_report_latest_{today_date}.html'
    with open(os.path.join('reports', filename), 'w') as f:
        f.write(html_content)

    print(f"HTML report generated: reports/{filename}")


generate_html_report()